In [199]:
import pandas as pd
import os
import csv
import xlrd
import fileinput
import sys
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
import re
import random
from random import sample ## random choose samples
from math import floor
from datetime import datetime

In [269]:
# read in CSV files
def read_in_files(year_pair):
    
    '''read in files and return the array of dataframe
    '''
    data_folder = 'Data/'
#     extention = '.xls'
    common_name = 'production'
    csv_ext = '.csv'
    year_dfs = {}
    dfs =[]  
    mix_year,max_year = year_pair ## get min,max year 
    for year in range(mix_year,max_year):
        file_name = str(year) + common_name + csv_ext
        file = os.path.join(data_folder,file_name)  
        df = pd.read_csv(file)
        df.columns = df.columns.to_series().apply(lambda x: x.strip()) ##strip the column names
        dfs.append(df)
    year_df = pd.concat(dfs,sort = False)
    #     print(f'first df has the column length of {result.columns}{len(result.columns)}')

    return year_df
    

In [274]:
wells01_df = read_in_files((2015,2016))  ## 1985,2012 has the same column names

In [275]:
wells01_df.tail()

,YEAR,API,OPERATOR,County,Horizontal Well Indicator,GAS_PRODUCTION_JANUARY,GAS_PRODUCTION_FEBRUARY,GAS_PRODUCTION_MARCH,GAS_PRODUCTION_APRIL,GAS_PRODUCTION_MAY,...,NGL_PROD_APR,NGL_PROD_MAY,NGL_PROD_JUN,NGL_PROD_JUL,NGL_PROD_AUG,NGL_PROD_SEP,NGL_PROD_OCT,NGL_PROD_NOV,NGL_PROD_DEC,NGL_PROD_TOTAL
56670,2015,8509077,"ZICKEFOOSE, R. L. AND W.F.",Ritchie,NaN,91,86,85,95,103,...,0,0,0,0,0,0,0,0,0,0
56671,2015,8509078,"ZICKEFOOSE, R. L. AND W.F.",Ritchie,NaN,111,106,104,117,126,...,0,0,0,0,0,0,0,0,0,0
56672,2015,8509079,"ZICKEFOOSE, R. L. AND W.F.",Ritchie,NaN,43,41,40,45,48,...,0,0,0,0,0,0,0,0,0,0
56673,2015,8509098,"ZICKEFOOSE, R. L. AND W.F.",Ritchie,NaN,48,46,45,50,54,...,0,0,0,0,0,0,0,0,0,0
56674,2015,8703626,"ZICKEFOOSE, R. L. AND W.F.",Roane,NaN,5,5,5,5,5,...,0,0,0,0,0,0,0,0,0,0


In [9]:
def print_iterator(it):
    for x in it:
        print(x, end=' ')
    print('')  # for new line


In [3]:
## check which row has strange letter
# for i in range(1,len(col)+1):
# #     print(i)
    
#     new_col = map(float,col[i-1:i])
# #     print(new_col,i)
#     print_iterator(new_col)
#     print(i) ##53179 column has a empty letter,somehow strip is not worked
#     except ValueError:
#         print(f'{i} row data has been corputed')
        

In [276]:
def get_monthly_data(df,pair_year):
    '''
    get monthly data for oil and gas production
    '''
    oil= 'OIL'
    gas = 'GAS'
    lng = 'LNG'
    ngl = 'NGL'
    condensate ='CONDENSATE'
    water = 'WATER'
    middle='PRODUCTION'
    month_pair = {
    'JANUARY':1,
    'FEBRUARY':2,
    'MARCH':3,
    'APRIL':4,
    'MAY':5,
    'JUNE':6,
    'JULY':7,
    'AUGUST':8,
    'SEPTEMBER':9,
    'OCTOBER':10,
    'NOVEMBER':11,
    'DECEMBER':12
    }
    
    ##container for saving months data
    gas_oil_month_dfs = []
    for month in month_pair.keys():## iterate all ove the months
        if pair_year==(1985,2012):
            ##slicing the dataframe
            gas_oil_month = df[['API','YEAR',gas+'_'+middle+'_'+month,oil+'_'+middle+'_'+month]]
            ## add month as a column
            gas_oil_month['Month'] = df['YEAR'].astype('str') +'/' +  str(month_pair[month])
            ## rename the columns for preparing the concatation
            gas_oil_month.columns = ['Well ID','Year','Month gas (mmscf)','Month oil (bbl)','Month (yyyymm)']
        elif pair_year==(2012,2013):
            gas_oil_month = df[['WELL_API','PRODUCTION_YEAR',gas+'_'+middle+'_'+month,oil+'_'+middle+'_'+month]]
            gas_oil_month['Month'] = df['PRODUCTION_YEAR'].astype('str') +'/'+ str(month_pair[month])
            gas_oil_month.columns = ['Well ID','Year','Month gas (mmscf)','Month oil (bbl)','Month (yyyymm)']
        elif pair_year==(2013,2015): ## has lng production
            gas_oil_month = df[['API','YEAR',gas+'_'+middle+'_'+month,
                                            oil+'_'+middle+'_'+month, lng+'_'+middle[0:4]+'_'+month[0:3]]]
            gas_oil_month['Month'] = df['YEAR'].astype('str') +'/' +  str(month_pair[month])
            gas_oil_month.columns = ['Well ID','Year','Month gas (mmscf)','Month oil (bbl)','Month condensate (bbl)','Month (yyyymm)']       
        elif pair_year==(2015,2016): ## has horizontial indicator
           
            gas_oil_month = df[['API','YEAR','Horizontal Well Indicator',gas+'_'+middle+'_'+month,
                                            oil+'_'+middle+'_'+month, ngl+'_'+middle[0:4]+'_'+month[0:3]]]
            gas_oil_month['Month'] = df['YEAR'].astype('str') +'/' +  str(month_pair[month])
            gas_oil_month.columns = ['Well ID','Year','Well_type','Month gas (mmscf)','Month oil (bbl)',
                                     'Month condensate (bbl)','Month (yyyymm)']    
        elif pair_year==(2016,2018):## add water column and condensate conlumns
            gas_oil_month = df[['API','Year','Horizontal Well Indicator',gas.title()+'_'+month.title(),
                                oil.title()+'_'+month.title(),water.title()+'_'+month.title(), 
                                condensate.title()+'_'+month.title()]]
            gas_oil_month['Month'] = df['Year'].astype('int').astype('str')+ '/'+  str(month_pair[month])
            gas_oil_month.columns = ['Well ID','Year','Well_type','Month gas (mmscf)','Month oil (bbl)',
                                     'Month condensate (bbl)','Month water (bbl)','Month (yyyymm)'] 
            
        elif pair_year==(2018,2019):
            
            gas_oil_month = df[['API','Year','Well_type',month[0:3].title()+'_'+gas.title(),
                                month[0:3].title()+'_'+oil.title(),month[0:3].title()+'_'+water.title(), 
                                month[0:3].title()+'_'+ngl]]
            gas_oil_month['Month'] = df['Year'].astype('int').astype('str')+ '/'+ str(month_pair[month]) 
            gas_oil_month.columns = ['Well ID','Year','Well_type','Month gas (mmscf)','Month oil (bbl)',
                                     'Month condensate (bbl)','Month water (bbl)','Month (yyyymm)'] 
        else:
            print(f'{year_pair} is out of range')

        gas_oil_month_dfs.append(gas_oil_month)
    
    result = pd.concat(gas_oil_month_dfs)
    return result

In [78]:
preprocessed_wells_group = get_monthly_data(wells05_df,(2018,2019))

C:\Users\zhangchunlei\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [217]:
def modify_to_standard(preprocessed_wells_group,pair_year):
    '''change the column names and covert the unit
        as well as add columns with none value if lack of
    '''
    standard_cols= ['Well ID','Well name','Month (yyyymm)','Monthly days','Month oil (bbl)','Month water (bbl)',\
                    'Month gas (mmscf)','Month condensate (bbl)','Month water injection (bbl)','Month gas injection (mmscf)',\
                    'Month steam injection (bbl)','Pool','Field']
    
    standard_df = pd.DataFrame()
    standard_df = preprocessed_wells_group 
    if pair_year==(1985,2012):
        selected_cols = ['Month (yyyymm)','Well ID','Month oil (bbl)','Month gas (mmscf)']
    elif pair_year==(2012,2013):
        selected_cols = ['Well ID','Year','Month gas (mmscf)','Month oil (bbl)','Month (yyyymm)']
    elif pair_year==(2013,2015): 
        selected_cols = ['Well ID','Year','Month gas (mmscf)','Month oil (bbl)','Month condensate (bbl)','Month (yyyymm)']
    elif pair_year==(2015,2016):
        selected_cols = ['Well ID','Year','Well_type','Month gas (mmscf)','Month oil (bbl)',
                                     'Month condensate (bbl)','Month (yyyymm)']  
    elif pair_year==(2016,2018):
        selected_cols =['Well ID','Year','Well_type','Month gas (mmscf)','Month oil (bbl)',
                                     'Month condensate (bbl)','Month (yyyymm)']  
    elif pair_year==(2018,2019):
        selected_cols =['Well ID','Year','Well_type','Month gas (mmscf)','Month oil (bbl)',
                                     'Month condensate (bbl)','Month (yyyymm)']  
    standard_df = standard_df[selected_cols]
    
    ## add column nonexisted comparing to standard_cols
    for col in standard_cols:
        if col not in  standard_df.columns:
              standard_df[col] = None
    ## unit convert for gas mcf
#     if standard_df['Month gas (mmscf)'].dtype != 'object':
    standard_df['Month gas (mmscf)'] = standard_df['Month gas (mmscf)'].astype('str').\
                                        apply(lambda x:x.strip().replace(',','')).map(float)
    standard_df['Month gas (mmscf)'] = standard_df['Month gas (mmscf)'].div(1000).round(6)
  
        
    ## slcing the df
    result = standard_df[standard_cols]
    
    return result
    

In [277]:
proprcessed_dfs =[]
def main():
    
#     proprcessed_dfs = []
    ## year_pairs to group dataframe together,since different years have dif column names
    year_pairs = [(1985,2012),(2012,2013),(2013,2015),(2015,2016),(2016,2018),(2018,2019)]
    ## read in files and saved as dataframes
    for year_pair in year_pairs:
        year_df= read_in_files(year_pair)
        preprocessed_wells_group = get_monthly_data(year_df,year_pair)
        modified_df =  modify_to_standard(preprocessed_wells_group,year_pair)
        proprcessed_dfs.append(modified_df)
        print(f'{year_pair} data has been preprocessed')
#         message = save_preprocessed_df_to(preprocessed_wells_group)##

    

In [278]:
main()

C:\Users\zhangchunlei\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(1985, 2012) data has been preprocessed


C:\Users\zhangchunlei\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(2012, 2013) data has been preprocessed


C:\Users\zhangchunlei\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

(2015, 2016) data has been preprocessed


C:\Users\zhangchunlei\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(2016, 2018) data has been preprocessed


C:\Users\zhangchunlei\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(2018, 2019) data has been preprocessed


In [220]:
proprcessed_dfs[0].to_csv('well1982-2012.csv')

In [17]:
proprcessed_dfs[1].to_csv('well2012-2014.csv')

In [221]:
def static_wells(fielname):
    '''
    handle with degree with 0,values,and convert to decimical degrees
    generate the standard column names
    '''
    
    data_foler = 'Data/'
    standar_cols = ['Country','State','Basin','Field','Latitude','Longitude','Well ID','Well name']
    ##read in data
    file = os.path.join(data_foler,fielname)
    df = pd.read_excel(file)
    ## drop first col
    df.drop(['Unnamed: 0'],axis = 1,inplace =True)
    ## lat an long with 0
    mask_lat = (df['WELL_HEAD_LATITUDE_DEGREES'] <= 1)## lat has positive value
    mask_long = (df['WELL_HEAD_LONGITUDE_DEGREES'] >=1) ##long has negative value
#     ##set those column with none value
#     df[mask_lat]['WELL_HEAD_LATITUDE_DEGREES'] =None
#     df[mask_lat]['WELL_HEAD_LATITUDE_MINUTES'] =None   
#     df[mask_lat]['WELL_HEAD_LATITUDE_SECONDS'] =None
#     df[mask_long]['WELL_HEAD_LONGITUDE_DEGREES'] =None
#     df[mask_long]['WELL_HEAD_LONGITUDE_MINUTES']  =None
#     df[mask_long]['WELL_HEAD_LONGITUDE_SECONDS'] = None
    ## get the masked left with long and lat none,and bigger > standard
    df_left = df[~(mask_lat | mask_long)]
#     print(df_left.head())
#     print(len(df_left))
    df_left['lat'] = df_left['WELL_HEAD_LATITUDE_DEGREES'] + df_left['WELL_HEAD_LATITUDE_MINUTES'].divide(60).round(6)\
                    + df_left['WELL_HEAD_LATITUDE_SECONDS'].divide(3600).round(6)
    df_left['long'] = df_left['WELL_HEAD_LONGITUDE_DEGREES']  - df_left['WELL_HEAD_LONGITUDE_MINUTES'].divide(60).round(6)\
                -  df_left['WELL_HEAD_LONGITUDE_SECONDS'].divide(3600).round(6)
    ##select the colmns
    selected = ['PERMIT_ID','lat','long']
    ## add states 
    result = df_left[selected]
    result['Country'] = 'UNITED STATES OF AMERICAN'
    result['State'] = 'WEST VIRGINIA'
    result['Well name'] = None
    result['Field'] = None
    result['Basin'] = None
    ## change column names in all
    result.columns = ['Well ID','Latitude','Longitude','Country','State','Well name','Field','Basin']
    result = result[standar_cols]
    
    return result

    
    

In [63]:
well_sta = static_wells('WellLocation.xlsx')

C:\Users\zhangchunlei\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\zhangchunlei\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\zhangchunlei\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

In [65]:
well_sta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103116 entries, 1 to 114258
Data columns (total 8 columns):
Country      103116 non-null object
State        103116 non-null object
Basin        0 non-null object
Field        0 non-null object
Latitude     91761 non-null float64
Longitude    91764 non-null float64
Well ID      103116 non-null int64
Well name    0 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 7.1+ MB


In [222]:
def  sample_wells(static,dynamics):
    '''
    chosen a number of wells and according to that wells
    generate well dynamics well dataframe and 
    static dataframe
    ------------------------------------------
    input:
    two dataframe
    output:
    well name and two selected dataframe
    ------------------------------------------
    '''
    portion = 0.15  ## portion of total wells been choosen
    random.seed = 32
    ## get dynmaics wells
    wells = list(dynamics['Well ID'].unique())
    ## choose number of wells
    chosen_number = floor(len(wells)*portion)  ## chose 1/5 of origianl well numbers
    chosen_wells = sample(wells, chosen_number)
    ##get dynamics dataframe
    chosen_dynamics = dynamics[dynamics['Well ID'].isin(chosen_wells)]
    ##get statics daframe
    chosen_statics = static[static['Well ID'].isin(chosen_wells)]
    
    
    return chosen_wells,chosen_dynamics,chosen_statics
    
    
    

In [170]:
chosen_wells,chosen_dynamics,chosen_statics = sample_wells(well_sta,wells_df)

In [223]:
def get_dynamics(chosen_wells,proprcessed_dfs):
    '''
    retrieve dynamics well data from preprocessed df
    -----------------------------------------------
    input: well names and original df list:lists
    out:df in the name of chosen_wells:dataframe
    --------------------------------------------
    '''
    length = len(proprcessed_dfs)
    ID_col_name = 'Well ID'
    years_dfs = []
    for i in range(1,length):
        mask = proprcessed_dfs[i][ID_col_name].isin(chosen_wells)
        years_df = proprcessed_dfs[i][mask]
        years_dfs.append(years_df)
    return years_dfs

In [172]:
dyn_wells = get_dynamics(chosen_wells,proprcessed_dfs)

In [224]:
def merge_df_together(chosen_dynamics,dyn_wells):
    '''
    merge dynamics dataframe together for all year pairs
    --------------------------------------------------
    input:two data sets of dataframe from years
          dtype:dataframe
    outpu:one merged datafraem
          dtype:dafarame
    ---------------------------------------------------
    '''
    concat_years = pd.concat(dyn_wells) ##concat well list first
    return pd.concat([chosen_dynamics,concat_years],sort =False)

In [166]:
dynamics_wells_com =  merge_df_together(chosen_dynamics,dyn_wells)

In [257]:
def str_for_time(date):
    '''
    output string format time
    '''
    new_date = datetime.strptime(date,'%Y/%m')
    str_time = new_date.strftime('%Y/%m')
    return str_time

In [283]:
def format_dyna(dynamics_wells_com):
    '''
    format month,ID columns
    --------------------------------------------------
    input:dataframe
    outpu::dafarame
    ---------------------------------------------------
    '''
    dynamics_wells_com['Month (yyyymm)'] = dynamics_wells_com['Month (yyyymm)'].apply(str_for_time)
    dynamics_wells_com.sort_values(by =['Well ID','Month (yyyymm)'],inplace =True)
    dynamics_wells_com['Well ID'] = dynamics_wells_com['Well ID'].astype('int')
    
    return dynamics_wells_com


In [284]:
def get_statics_and_dynamics():
    ## choose from 1985-2012 well production data, get the wells location
    wells_df = pd.read_csv('well1982-2012.csv')
    # proprcessed_dfs[0].to_csv('well1982-2012.csv')
    chosen_wells,chosen_dynamics,chosen_statics = sample_wells(well_sta,wells_df)
    ## according to the chosen wells select residual dynamics data
    dyn_wells = get_dynamics(chosen_wells,proprcessed_dfs)
    ##merg all dynamics data together
    dynamics_wells_com =  merge_df_together(chosen_dynamics,dyn_wells)
    ##format the dynamics_wells_com
    dynamics_wells_com_format = format_dyna(dynamics_wells_com)
    return chosen_wells,dynamics_wells_com_format,chosen_statics
    
    
    

In [285]:
chosen_wells,dynamics_wells_com,chosen_statics = get_statics_and_dynamics()

In [290]:
with open('well_names_chosen.csv','w') as f:
    f.write(chosen_wells)

TypeError: write() argument must be str, not list

In [289]:
dynamics_wells_com.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2348820 entries, 6 to 7460
Data columns (total 14 columns):
Unnamed: 0                     float64
Well ID                        int32
Well name                      object
Month (yyyymm)                 object
Monthly days                   object
Month oil (bbl)                object
Month water (bbl)              object
Month gas (mmscf)              float64
Month condensate (bbl)         object
Month water injection (bbl)    object
Month gas injection (mmscf)    object
Month steam injection (bbl)    object
Pool                           object
Field                          object
dtypes: float64(2), int32(1), object(11)
memory usage: 259.8+ MB


In [288]:
dynamics_wells_com[dynamics_wells_com['Well ID'] ==2102155]

,Unnamed: 0,Well ID,Well name,Month (yyyymm),Monthly days,Month oil (bbl),Month water (bbl),Month gas (mmscf),Month condensate (bbl),Month water injection (bbl),Month gas injection (mmscf),Month steam injection (bbl),Pool,Field
7715,7715.0,2102155,NaN,1985/01,NaN,0,NaN,0.092,NaN,NaN,NaN,NaN,NaN,NaN
1141547,7715.0,2102155,NaN,1985/02,NaN,0,NaN,0.057,NaN,NaN,NaN,NaN,NaN,NaN
2275379,7715.0,2102155,NaN,1985/03,NaN,0,NaN,0.066,NaN,NaN,NaN,NaN,NaN,NaN
3409211,7715.0,2102155,NaN,1985/04,NaN,0,NaN,0.035,NaN,NaN,NaN,NaN,NaN,NaN
4543043,7715.0,2102155,NaN,1985/05,NaN,0,NaN,0.040,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45451,NaN,2102155,None,2015/08,None,0,None,0.015,0,None,None,None,None,None
45451,NaN,2102155,None,2015/09,None,0,None,0.015,0,None,None,None,None,None
45451,NaN,2102155,None,2015/10,None,0,None,0.015,0,None,None,None,None,None
45451,NaN,2102155,None,2015/11,None,0,None,0.015,0,None,None,None,None,None
